In [1]:
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement

from joblib import Parallel, delayed                                                                                                                                

from tqdm import tqdm

In [2]:
index=False

In [4]:
"""
sudo docker run --name cassandra -p 127.0.0.1:9042:9042 -e CASSANDRA_CLUSTER_NAME=GangaTest -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra
"""
def cassandra_connection():
    """
    Connection object for Cassandra
    :return: session, cluster
    """
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect()
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS fakehealthcareorg
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """)
    session.set_keyspace('fakehealthcareorg')
    return session, cluster


In [5]:
session, cluster = cassandra_connection()

# Creating the keyspace

In [6]:
session.execute("""create keyspace dev
... with replication = {'class':'SimpleStrategy','replication_factor':1};""")

In [56]:
session.execute("""use dev;""")
session.execute("""DROP TABLE IF EXISTS JOB""")

In [57]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS JOB(
        id INT PRIMARY KEY, status varchar, name varchar, subjobs INT, application varchar,
        backend varchar, backend_actualCE varchar, comment varchar
    )""")
except Exception as e:
    print(e)

# Inserting the data:
1. 10K
2. 100K
3. 1000K

In [12]:
# 1. 10K
import pickle
jobs = pickle.load(open("rows.pkl", "rb"))
blobs = pickle.load(open("blobs.pkl", "rb"))

In [13]:
query = session.prepare("""
    INSERT INTO JOB(id, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (?,      ?,    ?,       ?,           ?,       ?,                ?,       ?)
    """)

if index:
    session.execute("""CREATE INDEX name_ind ON devices (name) USING CONSISTENCY ALL;""")


In [15]:
%%time
size = 100000
_jobs = jobs[:size]
_blobs = blobs[:size]
with tqdm(total=size) as progress:
    for i, (row, blob) in enumerate(zip(_jobs, _blobs)):
        row[0] = i+1
        blob['jid'] = i+1
        try:
            session.execute(query.bind(row))        
        except Exception as e:
            print(e)
        progress.update(1)

100%|██████████| 100000/100000 [01:09<00:00, 1446.08it/s]

CPU times: user 35.7 s, sys: 7.14 s, total: 42.9 s
Wall time: 1min 9s


In [38]:
from joblib import Parallel, delayed
rows = [[i]+row[1:] for i, row in enumerate(_jobs)]
def insert(data):
    flag = session.execute(
        query.bind(data)
    )
    return flag

In [58]:
%%time
result = Parallel(n_jobs=-1, verbose=1, backend="threading")(
    map(delayed(insert), rows)
)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

WriteTimeout: Error from server: code=1100 [Coordinator node timed out waiting for replica nodes' responses] message="Operation timed out - received only 0 responses." info={'consistency': 'LOCAL_ONE', 'required_responses': 1, 'received_responses': 0, 'write_type': 'SIMPLE'}

In [55]:
result[0].response_future

<ResponseFuture: query='<BoundStatement query="
    INSERT INTO JOB(id, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (?,      ?,    ?,       ?,           ?,       ?,                ?,       ?)
    ", values=[0, "'completed'", 'None', 1, "'85c74d55153c2b535aa07b2067c9381e'", "'Done'", "'LCG.CNAF.it'", "''"], consistency=Not Set>' request_id=22 result=None exception=None coordinator_host=127.0.0.1:9042>

In [36]:
%%time
future = session.execute_async("SELECT * FROM JOB")
rows = future.result()

CPU times: user 9.64 ms, sys: 3.62 ms, total: 13.3 ms
Wall time: 21.7 ms


In [37]:
%%time
len(list(rows))

CPU times: user 28 µs, sys: 5 µs, total: 33 µs
Wall time: 38.4 µs


0

In [26]:
%%time
rows = session.execute("SELECT * FROM JOB")

CPU times: user 14.2 ms, sys: 4.37 ms, total: 18.6 ms
Wall time: 60 ms


In [27]:
%%time
len(list(rows))

CPU times: user 362 ms, sys: 11.3 ms, total: 373 ms
Wall time: 1.01 s


100000